<a href="https://colab.research.google.com/github/odunayo12/data-wrangling-in-r-n-py/blob/master/01_tensor_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1dIpeujnom2vQwZBLXkSeAOrrQcME1IUr')

     |████████████████████████████████| 71kB 2.9MB/s 


# Hello `pytorch`, can I make you `function()`?


In this article, the goal is to introduce the following `pytorch` functions by demonstrating thier use in portfolio risk management. 

- `torch.ones()`
- `torch.diagonal()` 
- `torch.rand_like()`
- `torch.mv()` 
- `torch.transpose()`
- `torch.sum()` 

Before we begin, let's install and import PyTorch

In [ ]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy torch torchvision torchaudio

In [1]:
# Import torch and other required modules
import torch

In this tutorial we seek to explore the use of `pytorch` functions in solving for a portfolio optimization problem: The Markowitz Mean-Variance Portfolio
Theory or Modern portfolio Theory. The MPT is a diversification-driven investment management technique that seeks to maximize expected returns by allocating the investment amount in such a way that a risky asset equally pays a high return.
Assumptions have it that investors are risk averse. That is they will only be willign to taken-on more risk, so long it pays a high enough returns.
Thus more formally, given $n$ assets, our paremeter consist of  $$\begin{align}
  w &= \begin{bmatrix}
           w_{1} \\
           w_{2} \\
           \vdots \\
           w_{n}
    \end{bmatrix} \in \mathbb{R}^{n \times 1}, \quad
  m &= \begin{bmatrix}
           m_{1} \\
           m_{2} \\
           \vdots \\
           w_{n}
    \end{bmatrix} \in \mathbb{R}^{n \times 1}, \quad
  \Sigma &= \begin{bmatrix}
            a_{11} & 0 & 0& 0\\
            0 & a_{22} & 0& 0\\
            \vdots  & \vdots & \ddots & \vdots\\
            0 & 0 & 0& a_{nn}
    \end{bmatrix} \in \mathbb{R}^{n \times n}, \quad
 e &= \begin{bmatrix}
           1 \\
           1 \\
           \vdots \\
           1
         \end{bmatrix} \in \mathbb{R}^{n \times 1}.
  \end{align}$$ 
Where:
  1. $w$ is the vector of portfolion weights (Say I have \\$100, weights refers to the portion of my \$100 each asset class in my portfolio should take). Thus its only logical that it sums up to 1. That is, $\sum_{i=1}^nw_i = 1$;
  2. $\Sigma$ is the  covariance matrix for the returns on the assets in the portfolio;
  3. $m$ is a vector of expected returns.
  4. $e$ is a unit vector.
  5. Note that all matrix and vectors are in dimension $n$ of the number of assets in our portfolio. This tells us that the more the numer of assets in out porfolio, the more the computation required.

## Problem Statement.
Given the paremeter family above, we seek a porfolio weights that minimizes risk called the minimum variance porfolio. Thus, among all asset combinations we seek the one that yields minimum risk and maximizes profit given our investment sum.
This is stated as an optimization problem of the form:
$$\begin{align}
min \qquad w^{T}\Sigma w \\
s.t \qquad  w^{T} e = 1;
\end{align}$$
where $w^{T}\Sigma w$ is the variance of portfolio return. The problem above is satisfied or "solved" by:
$$\begin{align}
w= \frac{\Sigma^{-1}e}{e^{T} \Sigma^{-1}e}.
\end{align}$$

In the subsequent section we use applicable `pytorch` functions to simulate and solve a related problem.

## Function 1 -  `torch.ones()`
The `torch.ones(a,b)` function returns a tensor `X` of ones with dimension `a` by `b`. But if only a is supplied it returns an `1` by `4` vector of ones. the therefore create our unit vector $e$ using the function. We assume we have 4 assets in our portfolio. That is $n= 4.$ 

In [27]:
# Example 1 - working (change this)

e = torch.ones(4,1) # creates a 3 by 1 tensor
print(e)

tensor([[1.],
        [1.],
        [1.],
        [1.]])


Closing comments about when to use this function

Let's save our work using Jovian before continuing.

In [44]:
!pip install jovian --upgrade --quiet

In [45]:
import jovian

## Function 2 - `torch.diagonal()` and `torch.rand_like()`
For our covariance matrix $\Sigma$, which happens to be a diagonal matrix; the `torch.diagonal(a,b,...)` function comes in handy. The function, if `a,b,...` is a vector, generates a square matrix with `a,b,...` as the diagonal entries.

`torch.rand_like()`, on the other hand, is an extension of `torch.rand()`. Whereas the `torch.rand(a,b)` function generates a tensor of the random numbers from 0 upto but not including 1; `torch.rand_like()` returns random numbers that mimmick the size of tensor input. Lets see this in action

#### Example 1 - `torch.rand()`

In [7]:
y= torch.rand(3,2)
print(y)

tensor([[0.3570, 0.1294],
        [0.0239, 0.4440],
        [0.7853, 0.8482]])


#### Example 2 - `torch.rand_like()`

In [8]:
torch.rand_like(y)

tensor([[0.5361, 0.7656],
        [0.1740, 0.7561],
        [0.0566, 0.1665]])

As seen above, we need not specify dimension for the `torch.rand_like()` function. Its output rather "inherits" the dimension of its input.

In [9]:
torch.rand_like(y,1)

TypeError: ignored

Note that you cannot add increase the alter the dimension of the input vector in `torch.rand_like(y)` by specifying additional argument in it. If dimesion must be changed it has to be changed in the iput fuction. Suppose I want a 3-D tensor output, the following will do the magic.

In [10]:
y = torch.rand(3,2,1)
torch.rand_like(y)

tensor([[[0.9280],
         [0.9286]],

        [[0.8797],
         [0.0579]],

        [[0.7907],
         [0.8985]]])

#### Sigma $\Sigma$
In what follows, we combine `torch.rand_like()` with `torch.diag()` to generate a square matrix $\Sigma$ of dimension $e$.

In [28]:
rand_e = torch.rand_like(e)
transp_rand_e = rand_e.t() # t() is a shorthand for transpose()
transp_rand_e = transp_rand_e[0,:] #covert to python list so to make it compactible with the torch.diag() function
print("transp_rand_e", transp_rand_e)
sigma = torch.diag(transp_rand_e)
print(sigma)

transp_rand_e tensor([0.0237, 0.0014, 0.0792, 0.5202])
tensor([[0.0237, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0014, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0792, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.5202]])


Notice that `transp_rand_e`'s tensor constitutes the diagonal elements of `sigma`, which is the desired goal.

## Function 3 - `torch.inverse()`

`torch.inverse()` returns the inverse of the square matrix. Therefore, we will us it to derive the values of $\Sigma^{-1}$.






In [29]:
sigma_inverse = torch.inverse(sigma)

Notice that the input must be a square matrix, that is it must be a matrix of equal lenght and height.

In [13]:
inv_rand = torch.rand(4, 3)
inv_rand.inverse()

RuntimeError: ignored

The erroe above occurs because matrix `inv_rand` is a `4 x 3` in dimension.

## Function 4 `torch.transpose()`

`torch.transpose(a, b)` function returns the transpose of a matrix. That is, if matrix $A$ has dimension `a x b` the function returns a matrix of lenght `b x a`.

In what follows we use the function to generate the vector $e'$. We use the shorthand `t()`

In [30]:
e_transpose = e.t()[0,:] #convert to python list.
e_transpose

# uncomment to test.
# e.e_transpose()[0,:]

tensor([1., 1., 1., 1.])

## Function 5 - `torch.mv(a,b)` 

Performs a matrix-vector product of the matrix `a` and the vector `b`.

Below is the result for the value of the $ e^{T} \Sigma^{-1}$ part of the denominator of the desired weights $w$.

In [36]:
e_transpose_sigma_inverse = torch.mv(sigma_inverse,e_transpose)
e_transpose_sigma_inverse

tensor([ 42.2743, 694.1913,  12.6340,   1.9222])

The `@` symbol is a shorthand for matrix multiplication and `/` for scalar division. Notice that the quantity  $e^{T} \Sigma^{-1}e$ is scalar. Thus, the following gives its value.

In [42]:
e_transpose_sigma_inverse_e = e_transpose_sigma_inverse@e
e_transpose_sigma_inverse_e

tensor([751.0218])

Here as in everywhere else, the rule of matrix multiplication having conforming lenghts is applicable. The following code breaks because the matrix length are non-conforming

In [35]:
e@e_transpose_sigma_inverse

RuntimeError: ignored

Finally, the assets weights $w$ in our portfolio that minimizes the risk is given by the following quantity.

In [34]:
w = e_transpose_sigma_inverse/e_transpose_sigma_inverse_e
w

tensor([0.0563, 0.9243, 0.0168, 0.0026])

## Function 6 - `torch.sum()` 

`torch.sum()` as the name implies, retúrns the sum of the elements of tensor.

We will use this to check if our weights adds up to 1. If it does, then, hurray. Our simulation went well.

In [26]:
w.sum()

tensor(1.)

## Conclusion

As we can see, for a \$100 invstment sum, the model tels us to invest \$5.63 in Asset A, \$92.43 in Asset B, \$1.68 in asset C and \$0.26 in Asset C. 

**Disclaimer**: This is not intended to be an invstment advise.

In [47]:
jovian.commit(project='01-tensor-operations')

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/odunayo12/01-tensor-operations


'https://jovian.ai/odunayo12/01-tensor-operations'

## Reference Links

* Official documentation for tensor operations: https://pytorch.org/docs/stable/torch.html
